<a href="https://colab.research.google.com/github/ammarhusain/colab-sandbox/blob/master/save_restore_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save & Restore a Model

Save and Restore a model using TensorFlow v2. In this example, we will go over both low and high-level approaches: 
- Low-level: TF Checkpoint.
- High-level: TF Module/Model saver.

This example is using the MNIST database of handwritten digits as toy dataset
(http://yann.lecun.com/exdb/mnist/).


In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np
tf.enable_eager_execution()

In [0]:
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28

# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [0]:
# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [0]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

## 1) TF Checkpoint

Basic logistic regression

In [0]:
# Weight of shape [784, 10], the 28*28 image features, and total number of classes.
W = tf.Variable(tf.random.normal([num_features, num_classes]), name="weight")
# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name="bias")

# Logistic regression (Wx + b).
def logistic_regression(x):
    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Adam optimizer.
optimizer = tf.train.AdamOptimizer(learning_rate)

In [0]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

        # Compute gradients.
        gradients = g.gradient(loss, [W, b])

        # Update W and b following gradients.
        optimizer.apply_gradients(zip(gradients, [W, b]))

In [7]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

W0129 01:01:20.338340 140486231103360 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/math_grad.py:1394: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


step: 50, loss: 609.707092, accuracy: 0.636719
step: 100, loss: 365.773712, accuracy: 0.742188
step: 150, loss: 254.895569, accuracy: 0.800781
step: 200, loss: 177.209656, accuracy: 0.828125
step: 250, loss: 166.842133, accuracy: 0.875000
step: 300, loss: 205.821396, accuracy: 0.843750
step: 350, loss: 185.642197, accuracy: 0.855469
step: 400, loss: 187.432831, accuracy: 0.855469
step: 450, loss: 103.237244, accuracy: 0.910156
step: 500, loss: 105.639854, accuracy: 0.902344
step: 550, loss: 148.338531, accuracy: 0.871094
step: 600, loss: 118.066727, accuracy: 0.898438
step: 650, loss: 103.633392, accuracy: 0.882812
step: 700, loss: 118.955917, accuracy: 0.898438
step: 750, loss: 156.309814, accuracy: 0.871094
step: 800, loss: 114.063469, accuracy: 0.886719
step: 850, loss: 140.037262, accuracy: 0.871094
step: 900, loss: 101.334724, accuracy: 0.914062
step: 950, loss: 80.693604, accuracy: 0.914062
step: 1000, loss: 160.754852, accuracy: 0.871094


## Save and Load with TF Checkpoint

In [0]:
# Save weights and optimizer variables.
# Create a dict of variables to save.
vars_to_save = {"W": W, "b": b, "optimizer": optimizer}
# TF Checkpoint, pass the dict as **kwargs.
checkpoint = tf.train.Checkpoint(**vars_to_save)
# TF CheckpointManager to manage saving parameters.
saver = tf.train.CheckpointManager(
      checkpoint, directory="./tf-example", max_to_keep=5)

In [13]:
# Save variables.
saver.save()

'./tf-example/ckpt-1'

In [14]:
# Check weight value.
np.mean(W.numpy())

-0.07799074

In [0]:
# Reset variables to test restore.
W = tf.Variable(tf.random.normal([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [16]:
# Check resetted weight value.
np.mean(W.numpy())

-0.012396359

In [17]:
# Set checkpoint to load data.
vars_to_load = {"W": W, "b": b, "optimizer": optimizer}
checkpoint = tf.train.Checkpoint(**vars_to_load)
# Restore variables from latest checkpoint.
latest_ckpt = tf.train.latest_checkpoint("./tf-example")
checkpoint.restore(latest_ckpt)

In [18]:
# Confirm that W has been correctly restored.
np.mean(W.numpy())

-0.07799074

## 2) TF Model

Basic neural network with TF Model

In [0]:
from tensorflow.keras import Model, layers

In [0]:
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28

# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 100

In [0]:
# Create TF Model.
class NeuralNet(Model):
    # Set layers.
    def __init__(self):
        super(NeuralNet, self).__init__(name="NeuralNet")
        # First fully-connected hidden layer.
        self.fc1 = layers.Dense(64, activation=tf.nn.relu)
        # Second fully-connected hidden layer.
        self.fc2 = layers.Dense(128, activation=tf.nn.relu)
        # Third fully-connecter hidden layer.
        self.out = layers.Dense(num_classes, activation=tf.nn.softmax)

    # Set forward pass.
    def __call__(self, x, is_training=False):
        x = self.fc1(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
neural_net = NeuralNet()

In [0]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    y_true = tf.cast(y_true, tf.int64)
    crossentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return tf.reduce_mean(crossentropy)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Adam optimizer.
optimizer = tf.train.AdamOptimizer(learning_rate)

In [0]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = neural_net(x, is_training=True)
        loss = cross_entropy(pred, y)

        # Compute gradients.
        gradients = g.gradient(loss, neural_net.trainable_variables)

        # Update W and b following gradients.
        optimizer.apply_gradients(zip(gradients, neural_net.trainable_variables))

In [25]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = neural_net(batch_x, is_training=False)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 2.186938, accuracy: 0.917969
step: 200, loss: 2.180615, accuracy: 0.957031
step: 300, loss: 2.180062, accuracy: 0.957031
step: 400, loss: 2.181459, accuracy: 0.941406
step: 500, loss: 2.179266, accuracy: 0.953125
step: 600, loss: 2.180261, accuracy: 0.941406
step: 700, loss: 2.176317, accuracy: 0.968750
step: 800, loss: 2.176529, accuracy: 0.968750
step: 900, loss: 2.176332, accuracy: 0.972656
step: 1000, loss: 2.180584, accuracy: 0.941406


## Save and Load with TF Model

In [0]:
# Save TF model.
neural_net.save_weights(filepath="./tfmodel.ckpt")

In [29]:
# Re-build neural network model with default values.
neural_net = NeuralNet()
# Test model performance.
pred = neural_net(batch_x)
print("accuracy: %f" % accuracy(pred, batch_y))

accuracy: 0.085938


In [33]:
# Load saved weights.
neural_net.load_weights(filepath="./tfmodel.ckpt")

In [34]:
# Test that weights loaded correctly.
pred = neural_net(batch_x)
print("accuracy: %f" % accuracy(pred, batch_y))

accuracy: 0.941406
